In [1]:
import pandas as pd
import numpy as np

In [2]:
# Read Echonest Taste profile dataset
#taste = pd.read_csv('./data/train_triplets.txt', sep='\t', header=None, names=['user_id','song_id','play_count'])

In [3]:
#taste.head()

In [4]:
#taste.shape

In [5]:
import os
import pyspark

In [6]:
from pyspark import SparkContext
sc = SparkContext()

In [11]:
taste_file = os.path.join('.','data','train_triplets.txt')
taste_raw_data = sc.textFile(taste_file)

In [12]:
taste_raw_data

./data/train_triplets.txt MapPartitionsRDD[3] at textFile at NativeMethodAccessorImpl.java:0

In [14]:
taste_data = taste_raw_data.map(lambda line: line.split("\t")).map(lambda tokens: (tokens[0],tokens[1],tokens[2])).cache()

In [16]:
taste_data.take(5)

[(u'b80344d063b5ccb3212f76538f3d9e43d87dca9e', u'SOAKIMP12A8C130995', u'1'),
 (u'b80344d063b5ccb3212f76538f3d9e43d87dca9e', u'SOAPDEY12A81C210A9', u'1'),
 (u'b80344d063b5ccb3212f76538f3d9e43d87dca9e', u'SOBBMDR12A8C13253B', u'2'),
 (u'b80344d063b5ccb3212f76538f3d9e43d87dca9e', u'SOBFNSP12AF72A0E22', u'1'),
 (u'b80344d063b5ccb3212f76538f3d9e43d87dca9e', u'SOBFOVM12A58A7D494', u'1')]

In [30]:
from pyspark.sql import SQLContext, Row
sqlContext = SQLContext(sc)
#taste_rows = taste_data.map(lambda x: Row(user_id=x[0], song_id=x[1], play_count=int(x[2])))
taste_dataframe = sqlContext.createDataFrame(taste_data, ["user_id", "song_id", "play_count"])

In [31]:
taste_dataframe.take(5)

[Row(user_id=u'b80344d063b5ccb3212f76538f3d9e43d87dca9e', song_id=u'SOAKIMP12A8C130995', play_count=u'1'),
 Row(user_id=u'b80344d063b5ccb3212f76538f3d9e43d87dca9e', song_id=u'SOAPDEY12A81C210A9', play_count=u'1'),
 Row(user_id=u'b80344d063b5ccb3212f76538f3d9e43d87dca9e', song_id=u'SOBBMDR12A8C13253B', play_count=u'2'),
 Row(user_id=u'b80344d063b5ccb3212f76538f3d9e43d87dca9e', song_id=u'SOBFNSP12AF72A0E22', play_count=u'1'),
 Row(user_id=u'b80344d063b5ccb3212f76538f3d9e43d87dca9e', song_id=u'SOBFOVM12A58A7D494', play_count=u'1')]

In [33]:
# StringIndexer 
from pyspark.ml.feature import StringIndexer

userIndex = StringIndexer(inputCol="user_id", outputCol="user_index")
user_ID = userIndex.fit(taste_dataframe).transform(taste_dataframe)

NameError: name 'taste_dataframes' is not defined

In [18]:
training_RDD, validation_RDD, test_RDD = taste_data.randomSplit([6, 2, 2], seed=0L)
validation_for_predict_RDD = validation_RDD.map(lambda x: (x[0], x[1]))
test_for_predict_RDD = test_RDD.map(lambda x: (x[0], x[1]))

In [19]:
# Model selection (ALS parameter tuning)
from pyspark.mllib.recommendation import ALS
import math

seed = 5L
iterations = 10
regularization_parameter = 0.1
ranks = [4, 8, 12]
errors = [0, 0, 0]
err = 0
tolerance = 0.02

min_error = float('inf')
best_rank = -1
best_iteration = -1
for rank in ranks:
    model = ALS.train(training_RDD, rank, seed=seed, iterations=iterations, lambda_=regularization_parameter)
    predictions = model.predictAll(validation_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
    rates_and_preds = validation_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
    error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    errors[err] = error
    err += 1
    print 'For rank %s the RMSE is %s' % (rank, error)
    if error < min_error:
        min_error = error
        best_rank = rank

print 'The best model was trained with rank %s' % best_rank

Py4JJavaError: An error occurred while calling o60.trainALSModel.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 3.0 failed 1 times, most recent failure: Lost task 0.0 in stage 3.0 (TID 3, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/sommy/spark-2.1.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 174, in main
    process()
  File "/Users/sommy/spark-2.1.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 169, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/Users/sommy/spark-2.1.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 272, in dump_stream
    bytes = self.serializer.dumps(vs)
  File "/Users/sommy/spark-2.1.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 427, in dumps
    return pickle.dumps(obj, protocol)
  File "/Users/sommy/spark-2.1.0-bin-hadoop2.7/python/l